In [14]:
import os
import sys
import pandas as pd
import nltk
from tqdm.auto import tqdm
tqdm.pandas()

In [15]:
att = pd.read_csv('/h/haoran/projects/HurtfulWords/fill_in_blanks_examples/attributes.csv')

In [3]:
df = pd.read_pickle('/h/haoran/projects/BERT_DeBias/data/df_extract.pkl')

In [4]:
df.columns

Index(['category', 'chartdate', 'charttime', 'hadm_id', 'note_id', 'text',
       'subject_id', 'gender', 'dob', 'dod', 'fold', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'admittime', 'deathtime',
       'dischtime', 'hospital_expire_flag', 'discharge_location', 'adm_diag',
       'dod_merged', 'ethnicity_to_use', 'age_bin', '24h_mort', '48h_mort',
       '1mo_mort', '1yr_mort', '24h_disch', '48h_disch', 'die_in_hosp',
       'icd9_code', 'sents', 'sections', 'mod_text', 'toks', 'num_toks',
       'sent_toks_lens', 'icustay_id', 'age', 'oasis', 'oasis_prob', 'sofa',
       'sapsii', 'sapsii_prob', 'language_to_use'],
      dtype='object')

In [5]:
df.category.value_counts()

Nursing/other        821230
Radiology            378920
Nursing              220383
Physician            139763
ECG                  138160
Discharge summary     59652
Echo                  34036
Respiratory           31629
Nutrition              9361
General                8144
Rehab Services         5386
Social Work            2603
Case Management         940
Pharmacy                100
Consult                  98
Name: category, dtype: int64

In [6]:
df = df[df.category == 'Discharge summary']

In [7]:
df['toks'] = df['text'].progress_apply(lambda x: nltk.word_tokenize(x))

In [8]:
df['toks'] = df['toks'].apply(lambda x: list(map(lambda y: y.lower(), x)))

In [9]:
df.shape

(59652, 47)

In [10]:
for i in att.category.unique():
    df[i] = False

In [11]:
def cont(A, B): 
    n = len(A) 
    return any(A == B[i:i + n] for i in range(len(B)-n + 1)) 

In [12]:
for ind, i in att.iterrows():
    t = nltk.word_tokenize(i['attribute'])
    df[i['category']] = df[i['category']] | (df['toks'].apply(lambda x: cont(t, x)))
    print(t)

['a', 'low', 'pain', 'drug']
['paracetamol']
['nsaid']
['aspirin']
['advil']
['a', 'medium', 'pain', 'drug']
['codeine']
['tramadol']
['propoxyphene']
['buprenorphine']
['a', 'high', 'pain', 'drug']
['morphine']
['fentanyl']
['methadone']
['a', 'high', 'dosage', 'of', 'pain', 'killer']
['cvd']
['cad']
['chf']
['coronary', 'artery', 'disease']
['coronary', 'heart', 'disease']
['congestive', 'heart', 'failure']
['heart', 'failure']
['heart', 'attacks']
['heart', 'problems']
['cardiovascular', 'disease']
['htn']
['hypertension']
['high', 'blood', 'pressure']
['abnormally', 'high', 'bp']
['high', 'bp']
['pulmonary', 'artery', 'systolic', 'hypertension']
['diabetes']
['diabetes', 'mellitus']
['dnr']
['dni']
['etoh']
['heroin']
['marijuana']
['alcohol']
['suicidal', 'tendencies']
['schizophrenia']
['depression']
['anxiety']
['hallucinations']
['hiv']
['aids']


In [13]:
for i in att.category.unique():
    print(i, df[i].sum())

DRUG 36604
CVD 26719
HTN 35317
DIAB 15298
DNR 1103
ADD 31691
MENT 15025
HIV 2344


In [14]:
temp = df.groupby('subject_id').agg({i:any for i in att.category.unique()}).reset_index()

In [15]:
gender_map = df.set_index('subject_id')['gender'].to_dict()

In [16]:
temp['gender'] = temp['subject_id'].map(gender_map)

In [16]:
males = temp[temp.gender == 'M']
females = temp[temp.gender == 'F']

In [21]:
d = []
for i in att.category.unique():
    total = temp[i].sum()
    d.append((i, males[i].sum()/total, females[i].sum()/total))

In [4]:
for i in temp.columns:
    if i not in ['subject_id', 'gender']:
        print(i, temp[i].sum())

DRUG 27267
CVD 19422
HTN 26274
DIAB 10986
DNR 1056
ADD 24141
MENT 11539
HIV 1935


In [22]:
pd.DataFrame(d, columns = ['Category','Male Prev','Female Prev'])

,Category,Male Prev,Female Prev
0,DRUG,0.569113,0.430887
1,CVD,0.586809,0.413191
2,HTN,0.558347,0.441653
3,DIAB,0.563353,0.436647
4,DNR,0.518939,0.481061
5,ADD,0.573796,0.426204
6,MENT,0.484097,0.515903
7,HIV,0.645995,0.354005
